<a href="https://colab.research.google.com/github/SaketMunda/introduction-to-nlp/blob/master/nlp_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing with TensorFlow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq).

In [1]:
# Since we're going to experiment deep-learning models so we need to enable GPUs
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2c2e3fe3-ff59-0e8c-d3e9-d5763645af49)


## Get Helper functions

In [2]:
# Get helper_functions.py script from Github
!wget https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback

--2023-02-04 03:51:56--  https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2904 (2.8K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   2.84K  --.-KB/s    in 0s      

2023-02-04 03:51:57 (49.6 MB/s) - ‘helper_functions.py’ saved [2904/2904]



## Get a Text Dataset
The dataset that we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

See the original source here: https://www.kaggle.com/c/nlp-getting-started

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip the data
unzip_data('nlp_getting_started.zip')

--2023-02-04 03:51:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-02-04 03:51:59 (100 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing Text Dataset

To visualize our text samples, we first have to read them in, so we can do it through pandas.

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# check the shapes
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [5]:
# view some samples
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


So here, `text` is the tweet and `target` variable is to identify whether the tweet is a disaster or not, so if `1` then it's a disaster else not a disaster.

Let's visualize some random `training` samples, but before that this is a good practice to shuffle the training samples first,

In [6]:
train_df_shuffled = train_df.sample(frac=1, random_state=17)
# frac=1 means 100% of samples will be shuffled
train_df_shuffled

,id,keyword,location,text,target
7027,10072,typhoon,NaN,Typhoon Soudelor: When will it hit Taiwan ÛÒ ...,1
318,463,armageddon,NaN,RT @RTRRTcoach: #Love #TrueLove #romance lith ...,0
1681,2425,collide,www.youtube.com?Malkavius2,I liked a @YouTube video from @gassymexican ht...,0
5131,7318,nuclear%20reactor,"New York, New York",Japan's Restart of Nuclear Reactor Fleet Fast ...,1
2967,4262,drowning,"Hendersonville, NC",#ICYMI #Annoucement from Al Jackson... http://...,0
...,...,...,...,...,...
406,584,arson,"Jerusalem, Israel",Mourning notices for stabbing arson victims st...,1
5510,7863,quarantined,"Livonia, MI",Reddit's new content policy goes into effect m...,0
2191,3139,debris,NaN,Plane debris discovered on Reunion Island belo...,1
7409,10600,wounded,santo domingo,Police Officer Wounded Suspect Dead After Exch...,1


In [7]:
# how does the test set looks like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class ?
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# Let's visualize some random samples

import random
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(disaster)" if target > 0 else "(not a disaster)")
  print(f"Text:\n{text}\n")
  print("----------\n")

Target: 0 (not a disaster)
Text:
He just wrecked all of you http://t.co/y46isyZkC8

----------

Target: 0 (not a disaster)
Text:
I hope they fall off a cliff.

----------

Target: 1 (disaster)
Text:
Police Officer Wounded Suspect Dead After Exchanging Shots - http://t.co/iPHaZV47g7

----------

Target: 0 (not a disaster)
Text:
New post from @SeismicSoftware: 3 Major Challenges of Channel Sales Enablement http://t.co/kWMRCEkVTF

----------

Target: 0 (not a disaster)
Text:
You are listening to LLEGASTE TU - TWISTER EL REY

----------



## Split dataset into Train and Validation sets

Since the test set doesn't contain the target variable so we might need some unseen data for model to be validated after training, so how about splitting our training set for validating purpose with some amount.


In [10]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=17)

## Converting Text into Numbers

Our labels are in numerical form (0 and 1) but our tweets are in string form.

But machine learning algorithm learns only through numbers so we have to convert those tweets/texts into numbers.

In NLP, there are two main concepts for turning text into numbers,
- **Tokenization** : A straight mapping from **word**(known as *word-level tokenization*) or character(which is *character-level tokenization*) or sub-word(*sub-word tokenization*) to a numerical value. Just like One hot encoding, suppose we have a sentence as "My name is Alpha", then if we are mapping according to word, "My" would `0`, "name" as `1`, "is" as `2` and "Alpha" as `3`.
- **Embeddings** : An embedding is a representation of natural language which can be learned. Representation comes in the form of **feature-vector**. For example the word "Alpha" could be represented by 5-D vector `[0.564, 0.897, 0.456, -0.987, 0.15]`. The size of the feature vector is tuneable. There are two ways to use embeddings:

    - **Create your own embedding** - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as `tf.keras.layers.Embedding`) and an embedding representation will be learned during model training.
    - **Reuse a pre-learned embedding** - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.


Simply, 

**Tokenization** : Straight mapping from word to number.

**Embedding** : Richer representation of relationships between tokens.

It depends on your problem. You could try character-level tokenization/embeddings and word-level tokenization/embeddings and see which perform best. You might even want to try stacking them (e.g. combining the outputs of your embedding layers using [tf.keras.layers.concatenate](https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate)).

If you're looking for pre-trained word embeddings, [Word2vec embeddings](https://jalammar.github.io/illustrated-word2vec/), [GloVe embeddings](https://nlp.stanford.edu/projects/glove/) and many of the options available on TensorFlow Hub are great places to start.

Much like searching for a pre-trained computer vision model, we can search for pre-trained word embedding to use for your problem. Try searching for something like "use pre-trained word embeddings in TensorFlow".

### Text Vectorization

Mapping words to numbers.

To tokenize our words, we'll use the preprocessing layer,
`tf.keras.layers.preprocessing.TextVectorization`

In [11]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Using the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                 standardize="lower_and_strip_punctuation", # how to process the text
                                 split="whitespace", # how to split the text
                                 ngrams=None, # create groups of n-words
                                 output_mode='int', # how to map tokens to numbers
                                 output_sequence_length=None) # How long should the output sequence of tokens be?

About the above params,

- `max_tokens` : The maximum number of words in your vocabulary (e.g 20000 or the number of unique words in your text), includes a value for OOV(out of vocabulary) tokens
- `standardize` : Methods for standardizing text
- `split`: split the text
- `ngrams`: how many words to contain per token split, for example if 2, it splits tokens into continous sequences of 2
- `output_mode`: How to output tokens can be `int`(integer mapping), `binary`(OHE), `count` or `tf-idf`
- `output_sequence_length`: Length of tokenized sequence to output, For example if set to 150, all tokenized sequences will be 150 tokens long.

In the above cell, we have initialized the object with the default settings but let's customize it a little bit for our own use case.

In particular, let's set values for `max_tokens` and `output_sequence_length`.

For `max_tokens`(the number of words in the vocabulary), multiples of 10,000(`10,000`, `20,000`, `30,000`) or the exact number of unqiue words in your text(e.g `32,179`) are common values.

For our use case, `10,000`

And for the `output_sequence_length` we'll use the average number of tokens per Tweet in the training set. But first, we'll need to find it.

In [12]:
# Find average number of tokens (words) in training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# Setup text vectorization with custom variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

To map our `TextVectorization` instance `text_vectorizer` to our data, we can call the `adapt()` method on it whilst passing it our training set.

In [14]:
# fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

Training data mapped! Let's try our `text_vectorizer` on a custom sentence.

In [15]:
# create a sample sentence
sample_sentence = "There's a flood in my village!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[281,   3, 214,   4,  13, 881,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

Try our `text_vectorizer` on a few random sentences ?

In [16]:
import random
random_sentence = random.choice(train_sentences)
print(f'Original Text: \n{random_sentence}\
        \n\n Vectorized Text:')
text_vectorizer([random_sentence])

Original Text: 
#Tampa: Super Freestyle Explosion Live in Concert at Amalie Arena - Sep  19
? Ticket Info: http://t.co/ooGotO76uZ        

 Vectorized Text:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[2665, 1035,    1,  383,  206,    4, 1896,   17,    1,    1, 4653,
        1158, 3335, 1210,    1]])>

We can also check the unique tokens in our vocabulary using the `get_vocabulary()` method

In [17]:
# get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens

print(f"Number of words in Vocab:{len(words_in_vocab)}")
print(f"Top 5 most common words:{top_5_words}")
print(f"Bottom 5 least common words:{bottom_5_words}")

Number of words in Vocab:10000
Top 5 most common words:['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words:['ovo', 'overåÊhostages', 'overzero', 'overwatch', 'overturns']


### Embedding

**Create an Embedding using Embedding Layer**

We've got a way to map our text to numbers. How about we go a step further and turn those numbers into an embedding?

The powerful thing about an embedding is it can be learned during training. This means rather that just be static, a word's numeric representation can be improved as a model goes through data samples.

We can see what an embedding of a word looks like by using the `tf.keras.layers.Embedding` layer.



In [18]:
tf.random.set_seed(17)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length,
                             name="embedding_1")
embedding

`embedding` is a TensorFlow Layer, so that we can use it as part of a model, meaning its parameters(word representations) can be updated and improved as the model learns.



In [19]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text: \n{random_sentence}")
print("\n\nEmbedded version:")

# embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text: 
Does anyone know why #murfreesboro #walmart was evacuated this evening? @dnj_com


Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0256331 ,  0.03744579,  0.01614494, ...,  0.03304393,
          0.02807939,  0.01946756],
        [ 0.04895592,  0.00866304, -0.03608847, ...,  0.01149271,
         -0.04934025,  0.04239353],
        [-0.04007417, -0.00953885, -0.04488143, ..., -0.03326275,
         -0.03895334, -0.0359932 ],
        ...,
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049],
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049],
        [-0.04305307, -0.04987746, -0.0366137 , ...,  0.01739735,
          0.00826229,  0.04193049]]], dtype=float32)>

These values might not mean much to us but they're what our computer sees each word as. When our model looks for patterns in different samples, these values will be updated as necessary.

If we review the shape of Embedded Version Tensor it is, `(1, 15, 128)`, it means that,
- 1: Is the quantity of sequences(sentences) we passed
- 15: is the `max_length` that we decided to normalize every sentence, if greater than 15 tokens/words then trim extra tokens or if less than 15 then pad it.
- 128: is the array size of each words, for example above sentence as `Now`, for this token the embedding tensor will look like,

In [20]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 2.5633100e-02,  3.7445787e-02,  1.6144942e-02, -1.7866421e-02,
       -8.2955509e-04,  4.0926602e-02,  1.8598661e-03,  1.9636359e-02,
        4.7246207e-02, -1.2394834e-02,  3.3825804e-02,  1.0652147e-02,
       -3.6363374e-02,  3.5562981e-02, -8.8627711e-03, -1.7674305e-02,
       -4.6674158e-02, -2.0577395e-02,  4.7166098e-02,  3.6671232e-02,
       -3.5737611e-02, -1.9922828e-02,  3.8463440e-02,  3.6760569e-03,
       -1.2186907e-02, -4.7768153e-02, -5.4156780e-04, -4.4431090e-03,
       -3.8443375e-02, -3.0310441e-02, -4.1851975e-02,  1.2147427e-03,
        2.5584508e-02,  1.0067448e-03,  4.1332688e-02,  1.5695360e-02,
        4.0390160e-02, -9.1355927e-03, -2.0383179e-02, -8.6928383e-03,
       -3.0447340e-02, -4.0137090e-02, -3.3477068e-02, -1.8019736e-02,
        1.2825798e-02,  3.4661476e-02,  3.5838295e-02,  2.8555524e-02,
        3.0117955e-02,  4.5272652e-02, -2.6998771e-02,  3.3932064e-02,
       -3.8907230e-02,  2.565

## Evaluation function for our Model Experiments

Since we're going to perform multiple experiments by creating deeplearning models and scikit learn algorithms, so to track them we should create a common function for comparison.

There are various metrics to evaluate the classification models like precision, f1-score, recall. So let's look at them through a function altogether.

In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates Model accuracy, precision, recall and f1-score for a binary classification model
  """
  # calculate the model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)
  # calculate precision, recall, f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy * 100,
                   "precision":model_precision,
                   "recall": model_recall,
                   "f1-score": model_f1}
                  
  return model_results

## Modelling a Text Dataset

For experiments with various machine learning model for text classifier we will be considering below experiments:
- **Model 0** : Naive Bayes (baseline)
- **Model 1** : Feed-forward neural network (dense model) 
- **Model 2** : LSTM Model (RNN)
- **Model 3** : GRU (RNN)
- **Model 4** : Bidirectional-LSTM (RNN)
- **Model 5** : 1D CNN
- **Model 6** : TF Hub Pre-trained Feature Extractor
- **Model 7** : Same as model 6 with 10% of training samples

### Model 0 : Getting a baseline

We'll use `scikit-learn` library for building this model, and create a Scikit-Learn Pipeline using the TF-IDF (term frequency-inverse document frequency) formula to convert words into numbers and then model them using Multinomial Naive Bayes Algorithm.

> 📖 **Reading**: About TF-IDF on [Scikit-learn documentation](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

Multinomial Naive Bayes model is kind of shallow model which trains faster.

In [23]:
# evaluate the model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 80.18%


In [24]:
# let's make some predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0])

In [25]:
# it is similar to our labels
val_labels[:10]

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 1])

In [26]:
# get the results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 80.18372703412074,
 'precision': 0.8125567744156732,
 'recall': 0.8018372703412073,
 'f1-score': 0.7968681002825004}

### Model 1 : A Simple Dense Model

The first "deep" model we're going to build is a single layer dense model.

In [27]:
from tensorflow.keras import layers

# Create directory to save Tensorboard logs
SAVE_DIR = "model_logs"

# Build model with functional API
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn text inputs into numbers using text_vectorizer
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

# compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# fit the model
history_model_1 = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving Tensorboard log files to: model_logs/simple_dense_model/20230204-035205
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6152 - accuracy: 0.6854 - val_loss: 0.5256 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4439 - accuracy: 0.8167 - val_loss: 0.4640 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3478 - accuracy: 0.8616 - val_loss: 0.4596 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2846 - accuracy: 0.8864 - val_loss: 0.4721 - val_accuracy: 0.7979
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2380 - accuracy: 0.9101 - val_loss: 0.4938 - val_accuracy: 0.7953


In [28]:
# evaluate the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4938 - accuracy: 0.7953


[0.4938335716724396, 0.7952755689620972]

In [29]:
# make some predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.12096813],
       [0.00963379],
       [0.07517964],
       [0.43156   ],
       [0.9998043 ],
       [0.8580827 ],
       [0.13515036],
       [0.894733  ],
       [0.58565694],
       [0.08657645]], dtype=float32)

Alright ! Let's some more evaluations by using our common function for evaluation.

In [30]:
# for the evaluation we have to make it similar to our val_labels which is in 0 and 1
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 0., 1., 1., 0.], dtype=float32)>

In [31]:
simple_dense_results = calculate_results(y_true=val_labels,
                                         y_pred=model_1_preds)

simple_dense_results

{'accuracy': 79.52755905511812,
 'precision': 0.7966788703003602,
 'recall': 0.7952755905511811,
 'f1-score': 0.7932321923411083}

For comparing the results like these between two models, let's create a helper function

In [32]:
def compare_baseline_to_new_results(baseline_results, new_model_results):
  for k,v in baseline_results.items():
    print(f"Baseline {k}: {v:.2f}, New {k}: {new_model_results[k]:.2f}, Difference: {new_model_results[k] - v:.3f}")

In [33]:
compare_baseline_to_new_results(baseline_results, simple_dense_results)

Baseline accuracy: 80.18, New accuracy: 79.53, Difference: -0.656
Baseline precision: 0.81, New precision: 0.80, Difference: -0.016
Baseline recall: 0.80, New recall: 0.80, Difference: -0.007
Baseline f1-score: 0.80, New f1-score: 0.79, Difference: -0.004


## Visualizing Learned Embeddings

Our first model (`model_1`) contained an embedding layer (`embedding`) which learned a way of representing words as feature vectors by passing over the training data.

To understand what a text embedding is, let's visualize the embedding our model learned.

In [34]:
# get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

And now let's get our embedding layer's weights (these are the numerical representations of each word).

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [36]:
# get the weight matrix of embedding layer
embed_weights = model_1.layers[2].get_weights()[0] 
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)

(10000, 128)


In [37]:
embed_weights[0]

array([-0.05793183, -0.03914464, -0.02473383, -0.0040093 ,  0.02946341,
       -0.01301203,  0.05229729,  0.04887476,  0.06025089,  0.00805068,
        0.06344495,  0.06382315,  0.03528398, -0.0243746 , -0.03784521,
       -0.00860966, -0.029973  , -0.00939367,  0.05955042, -0.06665283,
       -0.01535041,  0.01206796,  0.0132463 ,  0.00885768, -0.03989456,
        0.00598109, -0.05867312,  0.01391944, -0.01342947, -0.03151253,
       -0.01170613, -0.01470301,  0.05553513,  0.06478364, -0.0330559 ,
        0.00180236,  0.03188962, -0.01603711,  0.0065343 , -0.0136434 ,
       -0.01092492,  0.03955517, -0.00912937, -0.04180072, -0.01730739,
       -0.00267582, -0.05257864, -0.0059381 , -0.00243136, -0.00406671,
        0.05934992,  0.00771909, -0.00551351, -0.03471248, -0.02665267,
       -0.01058052,  0.01899303, -0.05025221, -0.01186905, -0.00643936,
       -0.03974138, -0.02110856, -0.00443363, -0.0067405 ,  0.02272332,
        0.02054   , -0.01021376, -0.03714587,  0.03495811, -0.00

Now we've got these two objects, we can use the [Embedding Projector Tool](http://projector.tensorflow.org/) to visualize our embedding.

To use the embedding projector tool, we need two files,
- the embedding vectors (same as embedding weights)
- the metadata of the embedding vectors (the words they represent - our vocabulary)

In [38]:
# import io

# # Create output writers
# out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num==0:
#     continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file

# out_v.close()
# out_m.close()


# # Download files locally to upload to Embedding projector

# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("embedding_vectors.tsv")
#   files.download("embedding_metadata.tsv")

## Recurrent Neural Networks(RNN's)

For further experimentations we're going to use special kind of neural networks used for sequence data such as to predict the next location based on the prior location or may be to generate a new sequence based on the past sequences which is done through **Recurrent Neural Networks (RNN)**.

Recurrent Neural Networks can be used for a number of sequence-based problems:
- **One to One:** one input, one output, such as image classification
- **One to many:** one input, many output, such as image captioning
- **Many to one:** many inputs, one outputs, such as text classification
- **Many to Many:** many inputs, many outputs, such as machine translation (translating English to Spanish) or speech to text

Most commong RNN cell or layers used for designing the network are:
- LSTM (Long Short Term Memory)
- GRU (Gates Recurrent Unit)
- Bidirectional RNNs (passes forward and backward along a sequence, left to right and right to left)

The architecture of the RNNs would be,

      Input(text) -> Tokenize -> Embedding -> Layers -> Output (label probability)

### Model 2 : LSTM

> **Note**: For a best practice when we are comparing different models then embedding layer should be different because embedding layer is a learned representation of words, if we were to use the same embedding layer for each model, we'd be mixing what one model has learned with the next. 

In [39]:
# set random seed and creating embedding layer (new embedding layer for each model)
tf.random.set_seed(17)
from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                           output_dim=128,
                                           embeddings_initializer="uniform",
                                           input_length=max_length,
                                           name="embedding_2")

# Create LSTM model
input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = model_2_embedding(x)
#print(x.shape)
#x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (if we want to stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x)
output = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(input, output, name="model_2_LSTM")

According to the Tensorflow documentation on LSTM, it accepts 3D input tensor as [batch, timestamp, feature_vector] so when we stack one more cell of LSTM, then we must set `return_sequences=True` so that when next LSTM cell is stacked will be inject with 3D input or else it would through error of "expecting 3D tensor but received 2D"

### Helper Function for Compiling and Fitting the Model

In [40]:
# for compiling and fitting the model let's create a helper function

def compile_fit_RNNs(model, dir_name, experiment_name):
  """
  Function used for compiling and fitting the model and save the 
  tensorboard experiment as well on the passed directory

  Returns the model history
  """
  model.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

  # fit the model
  history = model.fit(train_sentences,
                      train_labels,
                      epochs=5,
                      validation_data=(val_sentences, val_labels),
                      callbacks=[create_tensorboard_callback(dir_name, experiment_name)])
  
  return history

In [41]:
model_2_history = compile_fit_RNNs(model=model_2, dir_name=SAVE_DIR, experiment_name="model_2_LSTM")

Saving Tensorboard log files to: model_logs/model_2_LSTM/20230204-035217
Epoch 1/5
215/215 [==============================] - 10s 18ms/step - loss: 0.5133 - accuracy: 0.7434 - val_loss: 0.4492 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3184 - accuracy: 0.8695 - val_loss: 0.4724 - val_accuracy: 0.7966
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2199 - accuracy: 0.9204 - val_loss: 0.5474 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1537 - accuracy: 0.9456 - val_loss: 0.6827 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1108 - accuracy: 0.9603 - val_loss: 0.8267 - val_accuracy: 0.7664


If we check the summary of the LSTM Model

In [42]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [43]:
# make some predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[0.02115482],
       [0.00182996],
       [0.01620475],
       [0.16472113],
       [0.9998977 ],
       [0.99410546],
       [0.01473758],
       [0.7996621 ],
       [0.8936984 ],
       [0.54785365]], dtype=float32)

In [44]:
# convert them to compare with labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 0., 1., 1., 1.], dtype=float32)>

In [45]:
# Let's calculate the results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 76.64041994750657,
 'precision': 0.7659025055163764,
 'recall': 0.7664041994750657,
 'f1-score': 0.7660201893659146}

In [46]:
# compare the baseline results with model_2
compare_baseline_to_new_results(baseline_results, model_2_results)

Baseline accuracy: 80.18, New accuracy: 76.64, Difference: -3.543
Baseline precision: 0.81, New precision: 0.77, Difference: -0.047
Baseline recall: 0.80, New recall: 0.77, Difference: -0.035
Baseline f1-score: 0.80, New f1-score: 0.77, Difference: -0.031


### Model 3 : GRU

The GRU cell has similar features to an LSTM cell but has less parameters.

In [47]:
# set the random seed and create new embedding layer
tf.random.set_seed(17)

model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name="embedding_3")

# Build GRU model
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3_GRU')


In [48]:
# let's see the summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

It's the same as LSTM model but with less params

In [49]:
# compile and fit the model
model_3_history = compile_fit_RNNs(model_3, SAVE_DIR, "model_3_GRU")

Saving Tensorboard log files to: model_logs/model_3_GRU/20230204-035238
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.5328 - accuracy: 0.7219 - val_loss: 0.4518 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3221 - accuracy: 0.8641 - val_loss: 0.4690 - val_accuracy: 0.8018
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2178 - accuracy: 0.9181 - val_loss: 0.5357 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1489 - accuracy: 0.9499 - val_loss: 0.6772 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1122 - accuracy: 0.9625 - val_loss: 0.6983 - val_accuracy: 0.7559


In [50]:
# make predictions
model_3_pred_probs = model_3.predict(val_sentences)
# convert them into labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs)) 
model_3_preds[:10]

24/24 [==============================] - 1s 4ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 0., 1., 1., 0.], dtype=float32)>

In [51]:
# calculate the result
model_3_results = calculate_results(y_true=val_labels,
                                   y_pred=model_3_preds)
model_3_results

{'accuracy': 75.59055118110236,
 'precision': 0.7551876442680979,
 'recall': 0.7559055118110236,
 'f1-score': 0.7551196015270819}

In [52]:
# compare with baseline score
compare_baseline_to_new_results(baseline_results, model_3_results)

Baseline accuracy: 80.18, New accuracy: 75.59, Difference: -4.593
Baseline precision: 0.81, New precision: 0.76, Difference: -0.057
Baseline recall: 0.80, New recall: 0.76, Difference: -0.046
Baseline f1-score: 0.80, New f1-score: 0.76, Difference: -0.042


It's looks like Baseline is still outperforming dense models, let's see one more model

### Model 4 : Bidirectional RNNs

A standard RNN will process a sequence from left to right, where as a bidirectional RNN will process the sequence from left to right and then again from right to left.

In [53]:
tf.random.set_seed(17)
model_4_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name="embedding_4")

# Build the Bidirectional Model
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name='model_4_Bidirectional')

In [54]:
# see the summary
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

Notice the parameters of the Bidirectional layer get doubled.

In [55]:
# compile and fit
model_4_history = compile_fit_RNNs(model_4, SAVE_DIR, "model_4_Bidirectional")

Saving Tensorboard log files to: model_logs/model_4_Bidirectional/20230204-035252
Epoch 1/5
215/215 [==============================] - 7s 13ms/step - loss: 0.5140 - accuracy: 0.7427 - val_loss: 0.4542 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3149 - accuracy: 0.8691 - val_loss: 0.4835 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 2s 12ms/step - loss: 0.2171 - accuracy: 0.9202 - val_loss: 0.5680 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1469 - accuracy: 0.9520 - val_loss: 0.6563 - val_accuracy: 0.7612
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1036 - accuracy: 0.9641 - val_loss: 0.7833 - val_accuracy: 0.7598


In [56]:
# make predictions
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 4ms/step


In [57]:
# convert them into labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 0., 1., 1., 0.], dtype=float32)>

In [58]:
# calculate the result
model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_preds)
model_4_results

{'accuracy': 75.98425196850394,
 'precision': 0.7600694650010854,
 'recall': 0.7598425196850394,
 'f1-score': 0.7599441744417845}

In [59]:
# compare with baseline scores
compare_baseline_to_new_results(baseline_results, model_4_results)

Baseline accuracy: 80.18, New accuracy: 75.98, Difference: -4.199
Baseline precision: 0.81, New precision: 0.76, Difference: -0.052
Baseline recall: 0.80, New recall: 0.76, Difference: -0.042
Baseline f1-score: 0.80, New f1-score: 0.76, Difference: -0.037


## Convolution Neural Networks for Text

CNNs can be used to train Text inputs by using the 1-Dimension layers instead of 2-Dimensional convolution like did with Images.

A Typical CNN architecture for sequences will look like the following:

```
Inputs (text) -> Tokenization -> Embedding -> Layers -> Outputs (class probabilities)
```

The main difference would be to use `tf.keras.layers.Conv1D()` instead of LSTM, GRU or Bidirectional cell in the layers, rest will be same.

### Model 5 : Conv1D 

Let's see first 1-dimensional convolutional layer (also called a temporal convolution), temporal means the data in timestamp sequences.

In [60]:
# Test our embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(['This is a test for conv layer'])) 
conv_1d = layers.Conv1D(filters=32, kernel_size=5, strides=1, activation="relu", padding="valid")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

The output shape of `conv_1d_output` is `[1,11,32]`, here the input length was `15` and output_dim - `128` of sequences in the embedding layer.

The 1-D conv layer compressed inline with its parameters. And the same goes for max pooling layer output.

Our text starts out as a string but gets converted to a feature vector of length 32 through various transformation steps(from tokenization to embedding to 1-Dimensional convolution to max pool).

In [61]:
## see the outputs of each layer
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
 array([[[ 0.00563604,  0.00831303, -0.03987605, ..., -0.00901501,
          -0.02458371, -0.00313252],
         [ 0.01050905,  0.07342832, -0.02361131, ..., -0.01137419,
           0.03161565,  0.01970429],
         [-0.03601195,  0.00866883, -0.04841169, ..., -0.04186256,
           0.03951383, -0.02277594],
         ...,
         [-0.05793183, -0.03914464, -0.02473383, ...,  0.00409454,
           0.02326122, -0.00015125],
         [-0.05793183, -0.03914464, -0.02473383, ...,  0.00409454,
           0.02326122, -0.00015125],
         [-0.05793183, -0.03914464, -0.02473383, ...,  0.00409454,
           0.02326122, -0.00015125]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
 array([[[0.        , 0.03461807, 0.04056317, 0.03943497, 0.03135176,
          0.00109527, 0.00164499, 0.02065373, 0.03384817, 0.06376338,
          0.        , 0.        , 0.        , 0.        , 0.02814319,
          0.        , 0.

Let's build the model

In [64]:
tf.random.set_seed(17)
from tensorflow.keras import layers

model_5_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer='uniform',
                                     input_length=max_length,
                                     name='embedding_5')

# Create 1-D conv layer
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name='model_5_Conv1D')

# see the summary of the model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [65]:
# compile and fit the model
model_5_history = compile_fit_RNNs(model_5, SAVE_DIR, "Conv1D_model_5")

Saving Tensorboard log files to: model_logs/Conv1D_model_5/20230204-041923
Epoch 1/5
215/215 [==============================] - 4s 10ms/step - loss: 0.5492 - accuracy: 0.7230 - val_loss: 0.4671 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3318 - accuracy: 0.8628 - val_loss: 0.4971 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1983 - accuracy: 0.9295 - val_loss: 0.5832 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1230 - accuracy: 0.9580 - val_loss: 0.7008 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0889 - accuracy: 0.9715 - val_loss: 0.7734 - val_accuracy: 0.7598


In [67]:
# make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
# convert them into labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

model_5_preds[:10]

24/24 [==============================] - 0s 4ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 1., 1., 0., 1., 1., 0.], dtype=float32)>

In [68]:
# calculate the results
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 75.98425196850394,
 'precision': 0.759153894646392,
 'recall': 0.7598425196850394,
 'f1-score': 0.7589128680335064}

In [69]:
# compare with baseline
compare_baseline_to_new_results(baseline_results, model_5_results)

Baseline accuracy: 80.18, New accuracy: 75.98, Difference: -4.199
Baseline precision: 0.81, New precision: 0.76, Difference: -0.053
Baseline recall: 0.80, New recall: 0.76, Difference: -0.042
Baseline f1-score: 0.80, New f1-score: 0.76, Difference: -0.038
